In [ ]:
from __future__ import print_function

import os
import json
import shutil
import sh
import yaml
from itertools import islice

import os
import glob
import sh
import yaml

from gitenberg import metadata
import jinja2

from second_folio import (GITENBERG_DIR, 
                          all_repos, 
                          apply_to_repos, 
                          travis_template, 
                          latest_epub,
                          new_travis_template,
                          repo_version,
                          )

from github_settings import (username, password)

REPOS_LIST = "/Users/raymondyee/C/src/gitenberg/Second-Folio/list_of_repos.txt"
GITENBERG_DIR = "/Users/raymondyee/C/src/gitenberg/"

METADATA_DIR = "/Users/raymondyee/C/src/gitenberg-dev/giten_site/metadata"
COVERS_DATA = "/Users/raymondyee/C/src/gitenberg/Second-Folio/covers_data.json"

In [ ]:
repos = all_repos
repos

In [ ]:
list(apply_to_repos(latest_epub, repos=repos))

# robobrowser

In [ ]:
from robobrowser import RoboBrowser

def post_to_yaml_loader(url, unglue_url="https://unglue.it/api/loader/yaml"):
        
    browser = RoboBrowser(history=True)
    browser.open(unglue_url)

    form = browser.get_forms()[0]
    form['repo_url'] = url
    # weird I have to manually set referer
    browser.session.headers['referer'] = unglue_url
    browser.submit_form(form)
    
    return browser

In [ ]:
b = post_to_yaml_loader('https://github.com/GITenberg/Adventures-of-Huckleberry-Finn_76/raw/master/metadata.yaml')
(b.url, b.response)

# opds

In [ ]:
from lxml import etree
import requests

opds_url = "https://unglue.it/api/opds/"
doc = etree.fromstring(requests.get(opds_url).content)
doc

# Failed attempt with requests to submit to yaml loader

In [ ]:
import requests

from lxml import etree
from lxml.cssselect import CSSSelector

unglue_url = "https://unglue.it/api/loader/yaml"

r = requests.get(unglue_url)
doc = etree.HTML(r.content)

sel = CSSSelector('input[name="csrfmiddlewaretoken"]')
csrftoken = sel(doc)[0].attrib.get('value')
csrftoken

In [ ]:
r = requests.post(unglue_url, 
                  data={'repo_url':
                          'https://github.com/GITenberg/Adventures-of-Huckleberry-Finn_76/raw/master/metadata.yaml',
                         'csrfmiddlewaretoken':csrftoken
                         },
                  headers={'referer':unglue_url})

In [ ]:
(r.status_code, r.content)

# travis webhooks

For https://travis-ci.org/GITenberg/Adventures-of-Huckleberry-Finn_76/builds/109712115 -- 2 webhooks were sent to <http://requestb.in/wrr6l3wr?inspect>:

* [Travis webhook #1 for https://travis-ci.org/GITenberg/Adventures-of-Huckleberry-Finn_76/builds/109712115](https://gist.github.com/rdhyee/7f33050732a09dfa93f3)
* [second webhook for https://travis-ci.org/GITenberg/Adventures-of-Huckleberry-Finn_76/builds/109712115](https://gist.github.com/rdhyee/8dc04b8fe52a9fefe3c2)

In [ ]:
import requests

raw_url_1 = (
 "https://gist.githubusercontent.com/rdhyee/7f33050732a09dfa93f3/raw/8abf5661911e7aedf434d464dd1a28b3d24d6f83/travis_webhook_1.json"
)

raw_url_2 = (
 "https://gist.githubusercontent.com/rdhyee/8dc04b8fe52a9fefe3c2/raw/8f9968f481df3f4d4ecd44624c2dc1b0a8e02a17/travis_webhook_2.json"
)

In [ ]:
r1 = requests.get(raw_url_1).json()
r2 = requests.get(raw_url_2).json()

In [ ]:
# url of metadata.yaml to load: 
# https://github.com/GITenberg/Adventures-of-Huckleberry-Finn_76/raw/master/metadata.yaml 

In [ ]:
r1.get('commit'), r1.get('repository', {}).get('name')


In [ ]:
r1

In [ ]:
r1.get('type'), r1['state'], r1['result'], r1.get('status_message')

In [ ]:
r2.get('type'), r2['state'], r2['result'], r2.get('status_message')

## travis webhook authentication

I think the documention is incorrect.  Instead of 'username/repository', just use the header `Travis-Repo-Slug`, which, I think, is just the full name of the repo -- e.g., `GITenberg/Adventures-of-Huckleberry-Finn_76`

When Travis CI makes the POST request, a header named Authorization is included. Its value is the SHA2 hash of the GitHub username (see below), the name of the repository, and your Travis CI token.

For instance, in Python, use this snippet:

```Python
from hashlib import sha256
sha256('username/repository' + TRAVIS_TOKEN).hexdigest()
```

Use this to ensure Travis CI is the one making requests to your webhook.

How to find `TRAVIS_TOKEN`?  You have to go your profile  (I thought you can use the travis CLI: `travis token` -- but that's for the "access token".  There are 3 different types of tokens in play for travis: [The Travis CI Blog: Token, Token, Token](https://blog.travis-ci.com/2013-01-28-token-token-token/))

So I'm waiting for <https://travis-ci.org/profile/rdhyee-GITenberg> to load up -- very slow on Chrome but fast on Firefox?



In [ ]:
sent_token = "6fba7d2102f66b16139a54e1b434471f6fb64d20c0787ec773e92a5155fad4a9"

from github_settings import TRAVIS_TOKEN, username
from hashlib import sha256


sha256('GITenberg/Adventures-of-Huckleberry-Finn_76' + TRAVIS_TOKEN).hexdigest()

# testing my webhook implementation

In [ ]:
import requests

url = "http://127.0.0.1:8000/api/travisci/webhook"

test_headers_url = \
   "https://gist.githubusercontent.com/rdhyee/a9242f60b568b5a9e8fa/raw/e5d71c9a17964e0d43f6a35bbf03efe3f8a7d752/webhook_headers.txt"

test_body_url = \
  "https://gist.githubusercontent.com/rdhyee/a9242f60b568b5a9e8fa/raw/e5d71c9a17964e0d43f6a35bbf03efe3f8a7d752/webook_body.json"
    
payload = requests.get(test_body_url).content

headers = dict([(k,v.strip()) for (k,v) in [line.split(":") for line in requests.get(test_headers_url).content.split('\n')]])

r = requests.post(url, data={'payload':payload}, headers=headers, allow_redirects=True)

(r.status_code, r.content)

In [ ]:
# example of a request to exercise exception

import json
payload = json.dumps({
  "repository":{  
      "id":4651401,
      "name":"Adventures-of-Huckleberry-Finn_76",
      "owner_name":"GITenberg",
      "url":"http://GITenberg.github.com/"
   },
  "status_message": "Passed",
  "type": "push"
})

r = requests.post(url, data={'payload':payload}, headers={}, allow_redirects=True)
(r.status_code, r.content)

In [ ]:
r = requests.get(url, allow_redirects=True)
(r.status_code, r.content)